In [3]:
%matplotlib inline
import pandas as pd 
import pandas_datareader
import datetime 
import matplotlib.pylab as plt 
import seaborn as sns
from matplotlib.pylab import style
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf 

style.use('ggplot')
plt.rcParams['font.sans-serif']=['SimHei']
plt.rcParams['axes.unicode_minus']=False


In [4]:
stockFile='data/T10yr.csv'
stock=pd.read_csv(stockFile,index_col=0,parse_dates=[0])
stock.tail(10) 

,Open,High,Low,Close,Volume,Adj Close
Date,,,,,,
2016-07-18,1.577,1.604,1.556,1.587,0,1.587
2016-07-19,1.553,1.575,1.546,1.558,0,1.558
2016-07-20,1.568,1.596,1.566,1.580,0,1.580
2016-07-21,1.601,1.628,1.563,1.565,0,1.565
2016-07-22,1.585,1.599,1.544,1.570,0,1.570
2016-07-25,1.584,1.584,1.554,1.571,0,1.571
2016-07-26,1.559,1.587,1.549,1.563,0,1.563
2016-07-27,1.570,1.570,1.511,1.515,0,1.515
2016-07-28,1.525,1.535,1.493,1.511,0,1.511
